In [1]:
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.chdir('../../')

env: CUDA_VISIBLE_DEVICES=0


In [ ]:
import torch
from functools import partial
from pointcept.engines.defaults import (
    default_argument_parser,
    default_config_parser,
    default_setup,
)
from pointcept.datasets import build_dataset_iseg, collate_fn_iseg

/data/ext_workspace/yuzijian2/.conda/envs/iseg3d_ptv3_2/lib/python3.8/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


### 1. config
pointcept 里加载 cfg 的流程

In [3]:
args = default_argument_parser().parse_args('')
args.config_file = 'configs/scannet/iseg-agile3d-v1m1.py'
args.options = {'save_path': 'exp/test/datasets_dataloader'}        # exp下只能两层，不然报错
args

Namespace(config_file='configs/scannet/iseg-agile3d-v1m1.py', dist_url='auto', machine_rank=0, num_gpus=1, num_machines=1, options={'save_path': 'exp/test/data_loader'})

In [4]:
cfg = default_config_parser(args.config_file, args.options)
cfg = default_setup(cfg)
cfg

Config (path: configs/scannet/iseg-agile3d-v1m1.py): {'weight': None, 'resume': False, 'evaluate': True, 'test_only': False, 'seed': 25924780, 'save_path': 'exp/test/data_loader', 'num_worker': 12, 'batch_size': 12, 'batch_size_val': None, 'batch_size_test': None, 'epoch': 200, 'eval_epoch': 100, 'clip_grad': None, 'sync_bn': False, 'enable_amp': True, 'amp_dtype': 'float16', 'empty_cache': False, 'empty_cache_per_epoch': False, 'find_unused_parameters': False, 'enable_wandb': True, 'wandb_project': 'AGILE3D', 'wandb_key': None, 'mix_prob': 0.0, 'param_dicts': None, 'semantic_ignore_label': -1, 'instance_ignore_label': -1, 'semantic_background_label': (0, 1), 'hooks': [{'type': 'CheckpointLoader'}, {'type': 'IterationTimer', 'warmup_iter': 2}, {'type': 'InformationWriter'}, {'type': 'ISegEvaluator', 'semantic_ignore': -1, 'instance_ignore': -1, 'semantic_background': (0, 1)}, {'type': 'CheckpointSaver', 'save_freq': 10}, {'type': 'PreciseEvaluator', 'test_last': False}], 'train': {'typ

### 2. dataset
构建 dataset, 可以使用固有的方法, 加载 data_root 中的数据, 还没做 transform
- 见 `pointcept/datasets/defaults.py: DefaultDataset.get_data()` 

In [6]:
train_dataset = build_dataset_iseg(cfg.data.train, ext_valid_assets=cfg.data.ext_valid_assets)
data_dict = train_dataset.get_data(0)
print(data_dict.keys())

[2025-05-19 05:05:42,333 INFO defaults.py line 70 2172258] Totally 1201 x 2 samples in scannet train set.


dict_keys(['normal', 'sampled_idx_fps_100', 'coord', 'color', 'name', 'split', 'segment', 'instance'])


### 3. train dataloader
加载经过 transform 的数据
- 见 `pointcept/datasets/defaults.py: DefaultDataset.__getitem__()` 

In [7]:
train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=3,
            shuffle=False,
            num_workers=1,
            sampler=None,
            collate_fn=collate_fn_iseg,
            pin_memory=True,
            worker_init_fn=None,
            drop_last=True,
            persistent_workers=True,
        )
data_iterator = enumerate(train_loader)
for idx, input_dict in data_iterator:
    break
input_dict.keys()

dict_keys(['coord', 'grid_coord', 'segment', 'instance', 'offset', 'feat'])

### 4. test dataloader
与 train 和 val 下的略有不同
- 见 `pointcept/datasets/defaults.py: DefaultDataset.prepare_test_data()`
- 会把各种 grid sampling 的 fragment 结果都收集起来
- TODO 现在跟 InstanceParser 有冲突，因为 instance parser 在 post_transform 中，所以暂时只用 train mode

In [7]:
test_dataset = build_dataset_iseg(cfg.data.test, ext_valid_assets=cfg.data.ext_valid_assets)
def base_collate_fn(batch):
    return batch
test_loader = test_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=1,
            shuffle=False,
            num_workers=1,
            pin_memory=True,
            sampler=None,
            collate_fn=base_collate_fn,
        )
data_iterator = enumerate(test_loader)
for idx, input_dict in data_iterator:
    break
input_dict[0].keys(), input_dict[0]["fragment_list"][0].keys()

[2025-05-19 05:15:14,771 INFO defaults.py line 70 2172258] Totally 312 x 1 samples in scannet val set.


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/data/ext_workspace/yuzijian2/.conda/envs/iseg3d_ptv3_2/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/data/ext_workspace/yuzijian2/.conda/envs/iseg3d_ptv3_2/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/data/ext_workspace/yuzijian2/.conda/envs/iseg3d_ptv3_2/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/data/shared_workspace/yuzijian/ws/ISeg3D/pointcept/datasets/defaults.py", line 187, in __getitem__
    return self.prepare_test_data(idx)
  File "/data/shared_workspace/yuzijian/ws/ISeg3D/pointcept/datasets/defaults.py", line 169, in prepare_test_data
    data_part_list = self.